In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# **Encoding**

In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# **Creating Training Examples**

In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for x,y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# **Building The Model**

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  model.build(input_shape=(BATCH_SIZE, None))
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, None, 256)             │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (64, None, 1024)            │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, None, 65)              │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

# **Creating a Loss Function**

In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 9.62107326e-04  2.55238777e-03  2.58322136e-04 ... -4.90181986e-03
   -1.02858001e-03 -1.32022717e-04]
  [ 1.03366352e-03  1.31895591e-03  6.09934237e-03 ... -8.75646062e-03
   -2.99333537e-04 -3.05108656e-03]
  [-5.76223386e-03  2.04400299e-03 -6.29303162e-04 ... -6.71098009e-03
    2.77748739e-04 -4.35758056e-03]
  ...
  [-6.03024149e-03  7.98759423e-03  4.86178510e-03 ... -8.23707506e-03
    2.50611478e-03 -3.58765852e-03]
  [-5.26363729e-03  5.38189663e-03  1.67782826e-03 ... -7.35613843e-03
    3.71264364e-03 -5.52970264e-03]
  [-1.81063428e-03  7.16607505e-03  3.10018822e-03 ... -1.01285353e-02
    3.15599586e-03 -3.85666382e-03]]

 [[ 4.50026570e-03 -5.43750683e-03  2.89326487e-03 ...  3.82275949e-03
   -9.76037700e-04  4.29864886e-04]
  [ 6.55702432e-04 -8.04024166e-04 -2.70096725e-03 ...  7.66490411e-04
   -1.74408767e-03 -7.57852511e-04]
  [-6.90737460e-03  8.35402228e-04 -8.83037131e-03 ...  1.35483034e-03
   -1.26926997e-03 -2.83379829e-03]
  ...
  [-7.496

In [16]:
pred=example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00096211  0.00255239  0.00025832 ... -0.00490182 -0.00102858
  -0.00013202]
 [ 0.00103366  0.00131896  0.00609934 ... -0.00875646 -0.00029933
  -0.00305109]
 [-0.00576223  0.002044   -0.0006293  ... -0.00671098  0.00027775
  -0.00435758]
 ...
 [-0.00603024  0.00798759  0.00486179 ... -0.00823708  0.00250611
  -0.00358766]
 [-0.00526364  0.0053819   0.00167783 ... -0.00735614  0.00371264
  -0.0055297 ]
 [-0.00181063  0.00716608  0.00310019 ... -0.01012854  0.003156
  -0.00385666]], shape=(100, 65), dtype=float32)


In [17]:
time_pred=pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 0.00096211  0.00255239  0.00025832 -0.00143754 -0.00265632  0.00058506
 -0.00161378  0.00223959 -0.00099285  0.008294    0.00227467 -0.00275118
 -0.00096078  0.0044377  -0.00672495  0.00207169  0.00113936  0.00202321
  0.00128931  0.00158908  0.00423499 -0.00753006 -0.00180641  0.00290694
  0.00072775  0.00306131  0.00255815 -0.00182416  0.00385401 -0.00108789
 -0.00417412 -0.002959   -0.00722601  0.00146578  0.00512989 -0.0008711
 -0.00204008  0.00029567 -0.00504463  0.0003568  -0.00603109  0.00491444
 -0.00501671 -0.00290687  0.00653206  0.00019825  0.00068665  0.00094066
  0.00183978 -0.000749    0.0017769  -0.00227429  0.0016972  -0.00094775
  0.00389626 -0.00587823  0.00557587  0.00245275  0.00060527 -0.00115768
  0.00176803 -0.00012672 -0.00490182 -0.00102858 -0.00013202], shape=(65,), dtype=float32)


In [18]:
sampled_indices=tf.random.categorical(pred,num_samples=1)
sampled_indices=np.reshape(sampled_indices,(1, -1))[0]
predicted_chars=int_to_text(sampled_indices)
predicted_chars

",A\n!nY-rkpVZh!S?UlYi\nzRtIvItA;WRbA?lCMSjjOnfLANgkc$k,VTdq-AC!w3W3,!h.YQfTxyGaLWe$rmr'GaDLC3FDb\nHic3q"

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# **Compiling the Model**

In [20]:
model.compile(optimizer='adam', loss=loss)

# **Creating Checkpoints**

In [22]:
checkpoints_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoints_dir, "ckpt_{epoch}.weights.h5") # Add '.weights.h5' to the filename

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# **Training**

In [23]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - loss: 2.8513
Epoch 2/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.8527
Epoch 3/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - loss: 1.5987
Epoch 4/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - loss: 1.4764
Epoch 5/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 23s 82ms/step - loss: 1.3995
Epoch 6/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - loss: 1.3465
Epoch 7/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 78ms/step - loss: 1.3052
Epoch 8/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - loss: 1.2664
Epoch 9/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - loss: 1.2324
Epoch 10/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 1.2014
Epoch 11/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 23s 84ms/step - loss: 1.1687
Epoch 12/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 19s 81ms/step - loss: 1.1379
Epoch 13/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - loss: 1.1018
Epoch 14/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step - loss: 1.0694
Epoch 15/50
172/172 ━━━━━━━━━

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [27]:
# Assume you want to load the checkpoint from epoch 50, modify the filename accordingly
checkpoint_path = os.path.join(checkpoints_dir, "ckpt_50.weights.h5")

# Create a new model instance with the desired batch_size (1 in this case)
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

# Load weights directly using the checkpoint path
model.load_weights(checkpoint_path)

In [29]:
checkpoint_num = 10
# Use keras.models.load_model to load the entire model along with the architecture
# or model.load_weights to load the weights alone.
checkpoint_path = os.path.join(checkpoints_dir, "ckpt_" + str(checkpoint_num) + ".weights.h5")
model.load_weights(checkpoint_path)  # Assuming 'model' is already built
# model.build(tf.TensorShape([1, None]))  # No need to build if using load_model

In [37]:
def generate_text(model, start_string):
  num_generate = 800

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature= 1.0

  # Remove or comment out the following line:
  # model.reset_states()

  for i in range(num_generate):
    predictions= model(input_eval)
    # The problem lies here. You are trying to squeeze dimension 0,
    # which has size 64 (batch size). Instead, you should be selecting
    # the first prediction from the batch using predictions[0]
    # predictions = tf.squeeze(predictions, 0)  # This line causes the error
    predictions = predictions[0]  # Select the first (and only) prediction from the batch
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])
  return(start_string + ''.join(text_generated))

In [39]:
inp= input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: venice
venice
Than what the flood of such a shrpendain reverend blood estables?--farewell.

GLOUCESTER:
My lord, Signior Gremio: there at this old
You must thy exflenge; not ambriaring
Her head in office, your mistake masse:
Mercy what so farmimy
Are not worn you tell you, achor the present battle.

Second Murderer:
Why, seek her not with curs and rdy
On resoluting a nipe the other, nath, it.
Thy night-new days not hate such edgects do this man
never report to please.

MENENIUS:
O woo'd to-morrow, not? the Antigones
Do could till you affair that I have leave them,
well all the first of womf in Coriolanus;
And whose suspicition of you of an ill clroud-ear,
That is a harm action: good my dear lords. Come,
Now claudio, thy lifes that were king, such as if, and ask.
CHEMARD II:
We lend them here and thy sl


In [45]:
import ipywidgets as widgets
from IPython.display import display

# Define function for play generator (example)
def generate_play(character_type):
    return f"Generated play with character: {character_type}"

# Create interactive widgets
character_selector = widgets.Dropdown(
    options=['Hero', 'Villain', 'Sidekick'],
    description='Character:'
)

generate_button = widgets.Button(description="Generate Play")
output = widgets.Output()

# Define button click behavior
def on_button_click(b):
    with output:
        print(generate_play(character_selector.value))

# Attach button click event
generate_button.on_click(on_button_click)

# Display widgets
display(character_selector, generate_button, output)



Dropdown(description='Character:', options=('Hero', 'Villain', 'Sidekick'), value='Hero')

Button(description='Generate Play', style=ButtonStyle())

Output()

In [47]:
# Save the model weights
model.save_weights('play_generator_weights.weights.h5')